## Sales Brochure Generator for Company

User will provide a company name and their main website url. Then the program will generate the brochure for the company.

### Steps to excuture (for me, the programmer)

- [ ] First scrape the data from the main website url
- [ ] Then find all the links from that website scraped content
- [ ] Then find all the relevent links from that website scraped content
- [ ] Then scrape the data from all the relevent links
- [ ] Then generate the brochure for the company with main website content + all the relevent link's content

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import time
import json
from IPython.display import display, Markdown

load_dotenv()

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [2]:
company_website_url = "https://huggingface.co" 

In [3]:
from scraper import fetch_website_links, fetch_website_contents

all_links_in_company_website = fetch_website_links(company_website_url)
company_website_content = fetch_website_contents(company_website_url)
print(all_links_in_company_website)

['https://huggingface.co/', 'https://huggingface.co/models', 'https://huggingface.co/datasets', 'https://huggingface.co/spaces', 'https://huggingface.co/docs', 'https://huggingface.co/enterprise', 'https://huggingface.co/pricing', 'https://huggingface.co/login', 'https://huggingface.co/join', 'https://huggingface.co/spaces', 'https://huggingface.co/models', 'https://huggingface.co/microsoft/VibeVoice-Realtime-0.5B', 'https://huggingface.co/Tongyi-MAI/Z-Image-Turbo', 'https://huggingface.co/zai-org/GLM-4.6V-Flash', 'https://huggingface.co/mistralai/Devstral-Small-2-24B-Instruct-2512', 'https://huggingface.co/zai-org/GLM-4.6V', 'https://huggingface.co/models', 'https://huggingface.co/spaces/Tongyi-MAI/Z-Image-Turbo', 'https://huggingface.co/spaces/mrfakename/Z-Image-Turbo', 'https://huggingface.co/spaces/dream2589632147/Dream-wan2-2-faster-Pro', 'https://huggingface.co/spaces/prithivMLmods/Qwen-Image-Edit-2509-LoRAs-Fast', 'https://huggingface.co/spaces/anycoderapps/VibeVoice-Realtime-0.

In [14]:
def find_relevent_links(comapny_website_url, links):

    print(len(links))
    response = openai_client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {
                "role": "system",
                "content": f'''
                We will create a sales broucher for a company, whose websit is {comapny_website_url}. We need to research first on this company.
                
                Here is the scraped data for this company website : {company_website_content}. 

               

                Go through this and then give me all the relevent links from {links}, for performing the research on this company.

                Response format will be a json like this:

                
                {{
                    "links": [ 
                        {{"type": "product page", "url": "https://somedomain.com/product"}},
                        {{"type": "about page", "url": "https://somedomain.com/about"}},
                        {{"type": "contact page", "url": "https://somedomain.com/contactinfo"}}
                    ]
                }}

                NOTE: Ignore homepage. And prioritize the important pages in the top items of the list.
                

                '''
            },
            {
                "role": "user",
                "content": "\n".join(links)  # this is basically converting the links list to a string
            }
        ],
        response_format={"type": "json_object"}
    )
    
    print(response.choices[0].message.content)
    return response.choices[0].message.content

In [15]:
relevent_links_json_string = find_relevent_links(company_website_url, all_links_in_company_website)

84
{
  "links": [
    {"type": "platform main page", "url": "https://huggingface.co/"},
    {"type": "models page", "url": "https://huggingface.co/models"},
    {"type": "datasets page", "url": "https://huggingface.co/datasets"},
    {"type": "spaces page", "url": "https://huggingface.co/spaces"},
    {"type": "documentation", "url": "https://huggingface.co/docs"},
    {"type": "enterprise solutions", "url": "https://huggingface.co/enterprise"},
    {"type": "pricing page", "url": "https://huggingface.co/pricing"},
    {"type": "model inference", "url": "https://huggingface.co/inference/models"},
    {"type": "endpoints info", "url": "https://huggingface.co/pricing#endpoints"},
    {"type": "spaces pricing", "url": "https://huggingface.co/pricing#spaces"},
    {"type": "company info", "url": "https://huggingface.co/huggingface"},
    {"type": "blog", "url": "https://huggingface.co/blog"},
    {"type": "community discussion", "url": "https://discuss.huggingface.co"},
    {"type": "compa

In [16]:
# Convert JSON string to Python dictionary
relevent_links_dict = json.loads(relevent_links_json_string)

print("Type:", type(relevent_links_dict))
print("\nParsed dictionary:")
print(relevent_links_dict)

Type: <class 'dict'>

Parsed dictionary:
{'links': [{'type': 'platform main page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'documentation', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise solutions', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'model inference', 'url': 'https://huggingface.co/inference/models'}, {'type': 'endpoints info', 'url': 'https://huggingface.co/pricing#endpoints'}, {'type': 'spaces pricing', 'url': 'https://huggingface.co/pricing#spaces'}, {'type': 'company info', 'url': 'https://huggingface.co/huggingface'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'company social media', 'url

In [17]:
system_prompt_for_brochure = """
    You're a sales brochure writer. You are given a list of links of a website and content main website (also the content of those relevent links).
    You will go through them a write a short, well formatted sales brochure for the company. 
    Target audiece is - customer, clients and investors.  

   



   Now Here is the website and links contents in the user prompt: 

    
"""

def generate_user_prompt_for_brochure():
    user_prompt = "## Landing page content: \n\n"
    user_prompt += company_website_content[:2000]
    # Access the 'links' array from the dictionary
    for item in relevent_links_dict['links']:
        print(item)
        user_prompt += f"\n\n## {item['type']}: \n\n"
        user_prompt += fetch_website_contents(item['url'])
    
    return user_prompt

user_prompt_for_brochure = generate_user_prompt_for_brochure()[:5000]
print(len(user_prompt_for_brochure))

{'type': 'platform main page', 'url': 'https://huggingface.co/'}
{'type': 'models page', 'url': 'https://huggingface.co/models'}
{'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}
{'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}
{'type': 'documentation', 'url': 'https://huggingface.co/docs'}
{'type': 'enterprise solutions', 'url': 'https://huggingface.co/enterprise'}
{'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}
{'type': 'model inference', 'url': 'https://huggingface.co/inference/models'}
{'type': 'endpoints info', 'url': 'https://huggingface.co/pricing#endpoints'}
{'type': 'spaces pricing', 'url': 'https://huggingface.co/pricing#spaces'}
{'type': 'company info', 'url': 'https://huggingface.co/huggingface'}
{'type': 'blog', 'url': 'https://huggingface.co/blog'}
{'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}
{'type': 'company social media', 'url': 'https://twitter.com/huggingface'}
{'type': 'github repository

In [18]:
response = openai_client.chat.completions.create(model="gpt-4.1-nano", messages=[
    {"role": "system", "content" : system_prompt_for_brochure},
    {"role": "user", "content": user_prompt_for_brochure}
])
display(Markdown(response.choices[0].message.content))

**Discover the Future of AI with Hugging Face — The Premier Platform for Machine Learning Innovation**

**About Us**  
Hugging Face stands at the forefront of artificial intelligence, serving as a vibrant community and a dynamic platform where machine learning enthusiasts, researchers, and enterprises collaborate to shape the future. Our mission is to create an open, accessible, and innovative environment for AI development, sharing, and deployment.

**What We Offer**  

- **Extensive Model Repository**  
Browse over **1 million models** spanning various tasks such as text generation, image processing, speech synthesis, and more. Our curated collection includes cutting-edge models like *microsoft/VibeVoice-Realtime-0.5B* for speech, *Tongyi-MAI/Z-Image-Turbo* for image enhancement, and advanced language models like *zai-org/GLM-4.6V*. Whether you're developing new applications or enhancing existing ones, find the perfect model to accelerate your project.

- **Versatile AI Applications & Spaces**  
Deploy and test AI models seamlessly using our **Spaces** platform, where real-time demos like image generation, video creation, and speech synthesis are readily available. For example, generate stunning images from text prompts or turn images into dynamic videos effortlessly. Our ecosystem supports all modalities—text, image, video, audio, and even 3D—empowering your creative potential.

- **Rich Dataset Library**  
Access and collaborate on **over 250,000 datasets** curated for diverse applications, from autonomous vehicle data to medical reasoning. Our datasets support rigorous training and evaluation, enabling accurate and responsible AI solutions.

- **Open Source & Collaboration**  
Host, share, and refine your models publicly, fostering a collaborative environment that speeds up innovation. Build your professional ML profile by showcasing your work to a global community, gaining peer feedback, and growing your network.

- **Enterprise & Accelerated Solutions**  
For organizations seeking to scale AI initiatives, we provide **paid computational resources**, custom enterprise solutions, and dedicated support. Move faster with our open-source stack and seamless integration options.

**Join Our Community**  
Whether you are a researcher, developer, startup, or enterprise, Hugging Face is your gateway to cutting-edge AI. Sign up today to explore, build, and collaborate on the future of machine learning.

**Get Started Now**  
- Explore **top trending models**  
- Access **powerful AI applications**  
- Collaborate with a thriving community  
- Accelerate your AI projects with enterprise solutions

Visit us at **huggingface.co** and become part of the AI revolution!